### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
19,41044721,79085,31214,51530,0,0,1073,0,2024-01-27 22:00,2024-01-27 22:00,1,4,4530,4,1
20,40853939,80120,79085,1345,11,65,1073,1,2024-01-28 21:58,2024-01-28 21:58,0,0,5607,3,1
21,40853940,79085,80120,1345,12,65,1073,4,2024-01-31 21:58,2024-01-31 21:58,2,1,7712,1,1
22,41048695,79085,47871,51530,0,0,1073,5,2024-02-01 22:00,2024-02-01 22:00,0,5,3047,4,1
23,41048973,72401,79085,3,0,0,1073,6,2024-02-02 18:27,2024-02-02 18:27,0,2,4274,3,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

5

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,79085,49173,31890,0,4,1,1,0,5-4-1a,8,9,9,41044721
1,31214,20778,38061,1,15,3,2,0,4222 new,12,10,12,41044721
0,80120,39199,31368,2,0,1,0,0,2017ofensive,7,10,8,40853939
1,79085,30756,38587,0,10,2,1,0,4-2-3-1a,9,9,10,40853939
0,79085,27429,39952,3,11,2,0,0,4-2-3-1a,9,9,8,40853940
1,80120,42527,30004,0,6,2,0,0,Mid3522020,7,8,5,40853940
0,79085,49502,29792,0,3,0,0,0,4-2-3-1a,8,8,10,41048695
1,47871,20661,40371,2,35,1,0,0,Série B vou eu,13,12,12,41048695
0,72401,47254,32048,0,1,1,0,0,442,7,7,5,41048973
1,79085,23151,38357,0,21,1,0,0,5-4-1a,8,8,7,41048973


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1038

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1345

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,6295,12,31,10,1,1,33,10,31523033010012,1345
1,6276,12,26,8,2,2,30,13,26517030008011,1345
2,89481,12,24,7,3,2,24,11,24513024007005,1345
3,79085,12,24,7,3,2,18,7,24511018007007,1345
4,89208,12,18,5,3,4,22,20,18502022005009,1345
5,78588,12,16,5,1,6,22,14,16508022005001,1345
6,80120,12,15,3,6,3,12,9,15503012003010,1345
7,121640,12,14,4,2,6,16,17,14499016004002,1345
8,121094,12,14,4,2,6,9,24,14485009004006,1345
9,122818,12,11,3,2,7,15,17,11498015003008,1345


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

3

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()